## Setup

In [1]:
%load_ext autoreload
%autoreload 2
%load_ext tensorboard

In [2]:
import pandas as pd
import numpy as np
from pydantic import BaseModel
import sys
import os
from lightning.pytorch.loggers import MLFlowLogger
from loguru import logger
from load_dotenv import load_dotenv
import time
import json
import torch
from torch.utils.data import DataLoader
import lightning as L
from lightning.pytorch.callbacks.early_stopping import EarlyStopping
from lightning.pytorch.callbacks import ModelCheckpoint
import mlflow

sys.path.insert(0, "..")

from src.utils.embedding_id_mapper import IDMapper
from src.algo.sequence.model import SequenceRatingPrediction
from src.algo.sequence.dataset import UserItemRatingDFDataset
from src.algo.sequence.trainer import SeqModellingLitModule
from src.eval.utils import create_rec_df, create_label_df, merge_recs_with_target
from src.eval.log_metrics import log_ranking_metrics, log_classification_metrics

In [3]:
load_dotenv(override = True)

True

In [4]:
class Args(BaseModel):
    testing: bool = False
    log_to_mlflow: bool = True
    experiment_name: str = "first-attempt"
    run_name: str = f"006-sequence-modelling"
    notebook_persit_dp: str = None
    
    user_col: str = "user_id"
    item_col: str = "parent_asin"
    rating_col: str = "rating"
    timestamp_col: str = "timestamp"
    group_name: str = "seq-modelling"

    top_K: int = 100
    top_k: int = 10

    batch_size: int = 64
    learning_rate: float = 0.001
    l2_reg: float = 1e-4
    early_stopping_patience: int = 10
    device: str = "cuda" if torch.cuda.is_available() else "cpu"
    max_epochs: int = 100

    # TwoTower specific
    dropout: float = 0.2
    embedding_dim: int = 128

    train_data_fp: str = os.path.abspath("../data_for_ai/interim/train_sample_interactions_16407u_neg_seq.parquet")
    val_data_fp: str = os.path.abspath("../data_for_ai/interim/val_sample_interactions_16407u_neg_seq.parquet")

    def init(self):
        self.notebook_persit_dp = os.path.abspath(f"data/{self.experiment_name}/{self.run_name}")

        if not (mlflow_uri := os.environ.get("MLFLOW_TRACKING_URI")):
            self.log_to_mlflow = False
            logger.warning("MLFlow is not enabled. Turn off tracking to Mlflow.")

        if self.log_to_mlflow:
            logger.info(
                f"Setting up Mlflow experiment: {self.experiment_name}, run_name: {self.run_name}"
            )

            self._mlf_logger = MLFlowLogger(
                experiment_name=self.experiment_name,
                run_name=self.run_name,
                tracking_uri=mlflow_uri,
                log_model=True,
            )

        if not self.testing:
            os.makedirs(self.notebook_persit_dp, exist_ok=True)
        return self
    
args = Args().init()
print(args.model_dump_json(indent=2))

2025-04-25 20:33:07.030 | INFO     | __main__:init:39 - Setting up Mlflow experiment: first-attempt, run_name: 006-sequence-modelling


{
  "testing": false,
  "log_to_mlflow": true,
  "experiment_name": "first-attempt",
  "run_name": "006-sequence-modelling",
  "notebook_persit_dp": "/home/dinhln/Desktop/real_time_recsys/notebooks/data/first-attempt/006-sequence-modelling",
  "user_col": "user_id",
  "item_col": "parent_asin",
  "rating_col": "rating",
  "timestamp_col": "timestamp",
  "group_name": "seq-modelling",
  "top_K": 100,
  "top_k": 10,
  "batch_size": 64,
  "learning_rate": 0.001,
  "l2_reg": 0.0001,
  "early_stopping_patience": 10,
  "device": "cuda",
  "max_epochs": 100,
  "dropout": 0.2,
  "embedding_dim": 128,
  "train_data_fp": "/home/dinhln/Desktop/real_time_recsys/data_for_ai/interim/train_sample_interactions_16407u_neg_seq.parquet",
  "val_data_fp": "/home/dinhln/Desktop/real_time_recsys/data_for_ai/interim/val_sample_interactions_16407u_neg_seq.parquet"
}


In [5]:
train_df = pd.read_parquet(args.train_data_fp)
val_df = pd.read_parquet(args.val_data_fp)

assert set(val_df[args.user_col].unique()).issubset(set(train_df[args.user_col].unique())), "Validation users must be present in training users."

assert set(val_df[args.item_col].unique()).issubset(set(train_df[args.item_col].unique())), "Validation items must be present in training items."
assert train_df[args.timestamp_col].max() < val_df[args.timestamp_col].min(), "Validation data must be after training data. Otherwise, its a data contamination problem."

In [6]:
train_df.head(3)

,user_id,parent_asin,rating,timestamp,user_indice,item_indice,item_sequence
0,AFZ4EK2LJ655XQKTEUELCARO6RYA,B00002EQCW,4.0,2003-01-23 03:28:15,8071,4,"[-1.0, -1.0, -1.0, -1.0, -1.0, -1.0, -1.0, -1...."
1,AFZ4EK2LJ655XQKTEUELCARO6RYA,B095JX15XF,0.0,2003-01-23 03:28:15,8071,4132,"[-1.0, -1.0, -1.0, -1.0, -1.0, -1.0, -1.0, -1...."
2,AFY2C4YOUP2SSMM43HD2L3FIEFZA,B00008SCFL,5.0,2003-11-25 18:12:09,7935,36,"[-1.0, -1.0, -1.0, -1.0, -1.0, -1.0, -1.0, -1...."


## Convert user_id and item_id into indices

In [7]:
# idm_path = os.path.abspath("../data_for_ai/interim/idm_16407u.json")
# idm = IDMapper().load(idm_path)
# idm.get_user_id(1)

In [8]:
# train_df = train_df.pipe(idm.map_indices)
# val_df = val_df.pipe(idm.map_indices)

# assert idm.unknown_item_index not in train_df["item_indice"].values, "Unknown item index must be present in training data."
# assert idm.unknown_user_index not in train_df["user_indice"].values, "Unknown user index must be present in training data."
# assert idm.unknown_item_index not in val_df["item_indice"].values, "Unknown item index must be present in validation data."
# assert idm.unknown_user_index not in val_df["user_indice"].values, "Unknown user index must be present in validation data."

In [9]:
# train_df.head(3)

In [10]:
# assert train_df.groupby(args.user_col)[args.item_col].nunique().min() >= 5, "Each user must have at least five items."
# assert train_df.groupby(args.item_col)[args.user_col].nunique().min() >= 10, "Each item must have at least ten users."

## Init model

In [11]:
def init_model(n_user, n_items, embedding_dim, dropout):
    """
    Initialize the model with the given parameters.
    """
    model = SequenceRatingPrediction(
        num_users = n_user,
        num_items = n_items,
        embedding_dim = embedding_dim,
        dropout = dropout,
    )
    return model

## Test

In [12]:
# embedding_dim = 8
# batch_size = 2
# hidden_dim = 8
# dropout = 0.2

# # Mock data
# user_indices = [0, 0, 1, 2, 2]
# item_indices = [0, 1, 2, 3, 4]
# timestamps = [0, 1, 2, 3, 4]
# ratings = [0, 4, 5, 3, 0]

# n_users = len(set(user_indices))
# n_items = len(set(item_indices))

# train_test_df = pd.DataFrame(
#     {
#         "user_indice": user_indices,
#         "item_indice": item_indices,
#         args.timestamp_col: timestamps,
#         args.rating_col: ratings,
#     }
# )

# model = init_model(n_users, n_items, embedding_dim, hidden_dim, dropout)

# # Example forward pass
# model.eval()
# user = torch.tensor([0])
# target_item = torch.tensor([2])
# predictions = model.predict(user, target_item)
# print(predictions)

In [13]:
# dataset = UserItemRatingDFDataset(
#     df = train_test_df,
#     user_col = "user_indice",
#     item_col = "item_indice",
#     rating_col = args.rating_col,
#     timestamp_col = args.timestamp_col,
# )

# train_loader = DataLoader(
#     dataset,
#     batch_size = batch_size,
#     shuffle = False,
#     drop_last= True
# )

In [14]:
# for batch_input in train_loader:
#     print(batch_input)

In [15]:
# # model
# lit_model = TwoTowerLitModule(model, log_dir=args.notebook_persit_dp, top_K = 3,top_k=3, idm = idm)
# # train model
# trainer = L.Trainer(
#     default_root_dir=f"{args.notebook_persit_dp}/test",
#     max_epochs=150,
#     accelerator="auto",
#     log_every_n_steps=1,
# )
# trainer.fit(
#     model=lit_model, train_dataloaders=train_loader, val_dataloaders=train_loader
# )

## Training loop

In [16]:
train_df

,user_id,parent_asin,rating,timestamp,user_indice,item_indice,item_sequence
0,AFZ4EK2LJ655XQKTEUELCARO6RYA,B00002EQCW,4.0,2003-01-23 03:28:15.000,8071,4,"[-1.0, -1.0, -1.0, -1.0, -1.0, -1.0, -1.0, -1...."
1,AFZ4EK2LJ655XQKTEUELCARO6RYA,B095JX15XF,0.0,2003-01-23 03:28:15.000,8071,4132,"[-1.0, -1.0, -1.0, -1.0, -1.0, -1.0, -1.0, -1...."
2,AFY2C4YOUP2SSMM43HD2L3FIEFZA,B00008SCFL,5.0,2003-11-25 18:12:09.000,7935,36,"[-1.0, -1.0, -1.0, -1.0, -1.0, -1.0, -1.0, -1...."
3,AFY2C4YOUP2SSMM43HD2L3FIEFZA,B00OQVZDJM,0.0,2003-11-25 18:12:09.000,7935,1859,"[-1.0, -1.0, -1.0, -1.0, -1.0, -1.0, -1.0, -1...."
4,AHF3TGIOSTD2UCHF3MO4MIHFJ5NQ,B0C68VS6LH,0.0,2004-06-18 02:02:57.000,13705,4773,"[-1.0, -1.0, -1.0, -1.0, -1.0, -1.0, -1.0, -1...."
...,...,...,...,...,...,...,...
254779,AES2U6KIAORYLTBPENQWMDVALTDQ,B07ZZVX1F2,5.0,2020-12-26 21:37:58.968,3109,3800,"[-1.0, 2237.0, 1691.0, 2694.0, 1633.0, 934.0, ..."
254780,AGU6SDEIMLBQZII2FVFJ6YIUZRKQ,B0BSF5LM3J,5.0,2020-12-26 22:29:54.459,11489,4622,"[107.0, 3997.0, 2858.0, 1680.0, 2919.0, 4109.0..."
254781,AGU6SDEIMLBQZII2FVFJ6YIUZRKQ,B0BZJ9BYZ3,0.0,2020-12-26 22:29:54.459,11489,4696,"[3997.0, 2858.0, 1680.0, 2919.0, 4109.0, 3695...."
254782,AG2HB7HEYSIAGYBEFFL666KVYTHA,B0895KGSY1,0.0,2020-12-26 23:06:03.454,8251,3896,"[-1.0, -1.0, 2531.0, 382.0, 2756.0, 3373.0, 34..."


In [17]:
rating_dataset = UserItemRatingDFDataset(
    train_df, "user_indice", "item_indice", args.rating_col, args.timestamp_col, "item_sequence"
)
val_rating_dataset = UserItemRatingDFDataset(
    val_df, "user_indice", "item_indice", args.rating_col, args.timestamp_col, "item_sequence"
)

train_loader = DataLoader(
    rating_dataset, batch_size=args.batch_size, shuffle=True, drop_last=True
)
val_loader = DataLoader(
    val_rating_dataset, batch_size=args.batch_size, shuffle=False, drop_last=False
)

In [18]:
for i in train_loader:
    print(i)
    break

{'user': tensor([ 8885,  4822,  1760, 13183,   161,  1146,  1475,  9830, 10495, 14473,
        13199,  9352, 15999,  4647, 15449,   122, 14539,  2529, 12417,  7073,
        10465,  5060, 16389, 12901,  1757,  5434,  9473, 12089,  3076,  3744,
         5539,  8145,   102,  3753,  1593, 10457, 15927,  3417, 11786,  4947,
        15035, 13413,  2342,  1927, 11992,  2960,   665, 15250,  3274,  2983,
         2932,  1210,  3547,   964,  8962,  2956,  3481,  5745, 11483,  7622,
         5679,   647,  9251, 13433]), 'item': tensor([2564, 2014, 3590, 4596, 1434, 2215, 4356, 2366, 1479, 3070, 3264, 1562,
        3828, 3400,  954, 4798, 1083, 4749, 4558, 3697, 1223, 1993, 3254, 1559,
        3154, 1640, 2359, 4793, 2871, 3919, 3700, 2419, 2399, 2106, 1859, 2570,
        3079, 4287, 4004, 1270, 3443,  432,  374, 1974, 3737, 3819, 3720, 2678,
        3441, 2540, 2124, 2053, 3580, 2691,  774, 1923, 4616, 1420, 3894, 3963,
        3743, 4594, 3919,  142]), 'rating': tensor([5., 0., 5., 5., 0., 1., 5

In [19]:
item_indices = train_df[args.item_col].unique()
user_indices = train_df[args.user_col].unique()
n_items = len(item_indices)
n_users = len(user_indices)

logger.info(f"Number of users: {n_users}, Number of items: {n_items}")
model = init_model(n_users, n_items, args.embedding_dim, args.dropout)

2025-04-25 20:33:08.954 | INFO     | __main__:<module>:6 - Number of users: 16407, Number of items: 4817


In [20]:
idm_path = os.path.abspath("../data_for_ai/interim/idm_16407u.json")
idm = IDMapper().load(idm_path)
idm.get_user_id(1)

'AE227WAM4NWQPJI33OPN7ZARNNZQ'

## Overfir 1 batch

In [21]:
early_stopping = EarlyStopping(
    monitor="val_loss", patience=5, mode="min", verbose=False
)

model = init_model(n_users, n_items, args.embedding_dim, args.dropout)
lit_model = SeqModellingLitModule(
    model,
    learning_rate=args.learning_rate,
    l2_reg=args.l2_reg,
    log_dir=args.notebook_persit_dp,
    accelerator=args.device,
    idm= idm
)

log_dir = f"{args.notebook_persit_dp}/logs/overfit"

# train model
trainer = L.Trainer(
    default_root_dir=log_dir,
    accelerator=args.device if args.device else "auto",
    max_epochs=300,
    overfit_batches=1,
    callbacks=[early_stopping],
)
trainer.fit(
    model=lit_model,
    train_dataloaders=train_loader,
    val_dataloaders=train_loader,
)
logger.info(f"Logs available at {trainer.log_dir}")

You are using the plain ModelCheckpoint callback. Consider using LitModelCheckpoint which with seamless uploading to Model registry.
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs
`Trainer(overfit_batches=1)` was configured so 1 batch will be used.
You are using a CUDA device ('NVIDIA GeForce RTX 3050 Laptop GPU') that has Tensor Cores. To properly utilize them, you should set `torch.set_float32_matmul_precision('medium' | 'high')` which will trade-off precision for performance. For more details, read https://pytorch.org/docs/stable/generated/torch.set_float32_matmul_precision.html#torch.set_float32_matmul_precision
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name  | Type                     | Params | Mode 
-----------------------------------------------------------
0 | model | SequenceRatingPrediction | 2.9 M  | train
-----------------------------------------------------------
2.9 M     Trainable params
0        

Sanity Checking: |          | 0/? [00:00<?, ?it/s]

/home/dinhln/Desktop/real_time_recsys/.venv/lib/python3.11/site-packages/lightning/pytorch/trainer/connectors/data_connector.py:252: UserWarning:

You requested to overfit but enabled val dataloader shuffling. We are turning off the val dataloader shuffling for you.

/home/dinhln/Desktop/real_time_recsys/.venv/lib/python3.11/site-packages/lightning/pytorch/trainer/connectors/data_connector.py:425: PossibleUserWarning:

The 'val_dataloader' does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` to `num_workers=19` in the `DataLoader` to improve performance.

/home/dinhln/Desktop/real_time_recsys/.venv/lib/python3.11/site-packages/lightning/pytorch/trainer/connectors/data_connector.py:252: UserWarning:

You requested to overfit but enabled train dataloader shuffling. We are turning off the train dataloader shuffling for you.

/home/dinhln/Desktop/real_time_recsys/.venv/lib/python3.11/site-packages/lightning/pytorch/trainer/conne

Training: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

...

Validation: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

2025-04-25 20:33:30.697 | INFO     | src.algo.sequence.trainer:on_fit_end:122 - Logging classification metrics...
2025-04-25 20:33:55.024 | INFO     | src.algo.sequence.trainer:on_fit_end:125 - Logging ranking metrics...


Generating recommendations:   0%|          | 0/4102 [00:00<?, ?it/s]

2025-04-25 20:38:34.256 | INFO     | src.algo.sequence.trainer:on_fit_end:128 - Evidently metrics are available at: /home/dinhln/Desktop/real_time_recsys/notebooks/data/first-attempt/006-sequence-modelling
2025-04-25 20:38:34.258 | INFO     | __main__:<module>:30 - Logs available at /home/dinhln/Desktop/real_time_recsys/notebooks/data/first-attempt/006-sequence-modelling/logs/overfit/lightning_logs/version_13


## Run on all data

In [24]:
early_stopping = EarlyStopping(
    monitor="val_loss", patience=args.early_stopping_patience, mode="min", verbose=False, min_delta=0.0025
)

checkpoint_callback = ModelCheckpoint(
    dirpath=f"{args.notebook_persit_dp}/checkpoints",
    filename="best-checkpoint",
    save_top_k=1,
    monitor="val_loss",
    mode="min",
)

model = init_model(n_users, n_items, args.embedding_dim, args.dropout)
lit_model = SeqModellingLitModule(
    model,
    learning_rate=args.learning_rate,
    l2_reg=args.l2_reg,
    log_dir=args.notebook_persit_dp,
    accelerator=args.device,
    idm= idm
)

log_dir = f"{args.notebook_persit_dp}/logs/run"

# train model
trainer = L.Trainer(
    default_root_dir=log_dir,
    accelerator=args.device if args.device else "auto",
    max_epochs=args.max_epochs,
    callbacks=[early_stopping, checkpoint_callback],
    logger=args._mlf_logger if args.log_to_mlflow else None,
)
trainer.fit(
    model=lit_model,
    train_dataloaders=train_loader,
    val_dataloaders=val_loader,
)

# Change the library as a workaround for the issue in the latest Lightning release
#https://github.com/Lightning-AI/pytorch-lightning/pull/20669/commits/429f732a0528c558e701da7ec01e51c1e2e4f32e

GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs
/home/dinhln/Desktop/real_time_recsys/.venv/lib/python3.11/site-packages/lightning/pytorch/callbacks/model_checkpoint.py:654: UserWarning:

Checkpoint directory /home/dinhln/Desktop/real_time_recsys/notebooks/data/first-attempt/006-sequence-modelling/checkpoints exists and is not empty.

LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name  | Type                     | Params | Mode 
-----------------------------------------------------------
0 | model | SequenceRatingPrediction | 2.9 M  | train
-----------------------------------------------------------
2.9 M     Trainable params
0         Non-trainable params
2.9 M     Total params
11.462    Total estimated model params size (MB)
10        Modules in train mode
0         Modules in eval mode


Sanity Checking: |          | 0/? [00:00<?, ?it/s]

/home/dinhln/Desktop/real_time_recsys/.venv/lib/python3.11/site-packages/lightning/pytorch/trainer/connectors/data_connector.py:425: PossibleUserWarning:

The 'val_dataloader' does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` to `num_workers=19` in the `DataLoader` to improve performance.

/home/dinhln/Desktop/real_time_recsys/.venv/lib/python3.11/site-packages/lightning/pytorch/trainer/connectors/data_connector.py:425: PossibleUserWarning:

The 'train_dataloader' does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` to `num_workers=19` in the `DataLoader` to improve performance.



Training: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

...

Validation: |          | 0/? [00:00<?, ?it/s]

2025-04-25 21:07:51.971 | INFO     | src.algo.sequence.trainer:on_fit_end:122 - Logging classification metrics...
2025-04-25 21:07:53.541 | INFO     | src.algo.sequence.trainer:on_fit_end:125 - Logging ranking metrics...


Generating recommendations:   0%|          | 0/606 [00:00<?, ?it/s]

2025-04-25 21:08:31.056 | INFO     | src.algo.sequence.trainer:on_fit_end:128 - Evidently metrics are available at: /home/dinhln/Desktop/real_time_recsys/notebooks/data/first-attempt/006-sequence-modelling


🏃 View run 006-sequence-modelling at: http://localhost:5002/#/experiments/9/runs/9fd559821e9b474380a208d237dbd510
🧪 View experiment at: http://localhost:5002/#/experiments/9


## Clean up

In [23]:
all_params = [args]

if args.log_to_mlflow:
    run_id = trainer.logger.run_id

    with mlflow.start_run(run_id=run_id):
        for params in all_params:
            params_dict = params.model_dump()
            params_ = dict()
            for k, v in params_dict.items():
                if k == "top_K":
                    k = "top_big_K"
                if k == "top_k":
                    k = "top_small_k"
                params_[f"{params.__repr_name__()}.{k}"] = v
            mlflow.log_params(params_)

🏃 View run 006-sequence-modelling at: http://localhost:5002/#/experiments/9/runs/9fd559821e9b474380a208d237dbd510
🧪 View experiment at: http://localhost:5002/#/experiments/9
